In [26]:
import pandas as pd
from gensim import models
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import *

In [27]:
trainDF = pd.read_csv("Data/train.csv")
testDF = pd.read_csv("Data/test.csv")

trainDF.question1.fillna('abc', inplace=True)
trainDF.question2.fillna('abc', inplace=True)
testDF.question1.fillna('abc', inplace=True)
testDF.question2.fillna('abc', inplace=True)

In [28]:
def cleanQuestion(question):
    """Functions to clean question pairs
    """
    # convert to lower case
    question = question.lower()
#     question = question.replace()
    question = question.decode('utf-8')
    stop = stopwords.words('english')
    stemmer = PorterStemmer()
    questions = re.sub(r'[^\x00-\x7F]+', ' ', question, flags=re.MULTILINE)
    question = ' '.join([word for word in question.split() if word not in stop])
    question = ' '.join([stemmer.stem(word) for word in question.split()])
    # remove extra spaces
    question = re.sub(r'[\s]+', ' ', question, flags=re.MULTILINE)
    # remove all punctuations
    question = re.sub(r'[^a-zA-Z]', ' ', question, flags=re.MULTILINE)
    return question

In [29]:
def getLabeledSentence(questions, label):
    labeledQuestions=[]
    for uid, line in enumerate(questions):
        labeledQuestions.append( models.doc2vec.LabeledSentence(words=line.split(), tags=[label+str(uid)]))
    
    return labeledQuestions

In [30]:
testDF.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [31]:
trainDF.question1 = trainDF.question1.apply(cleanQuestion)
trainDF.question2 = trainDF.question2.apply(cleanQuestion)

In [32]:
testDF.question1 = testDF.question1.apply(cleanQuestion)
testDF.question2 = testDF.question2.apply(cleanQuestion)

KeyboardInterrupt: 

In [33]:
testDF.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [34]:
labeledQuestions1 = getLabeledSentence(trainDF.question1.tolist(), 'question1')
labeledQuestions2 = getLabeledSentence(trainDF.question2.tolist(), 'question2')

KeyboardInterrupt: 

In [35]:
labeledQuestionstest1 = getLabeledSentence(testDF.question1.tolist(), 'testquestion1')
labeledQuestionstest2 = getLabeledSentence(testDF.question2.tolist(), 'testquestion2')

KeyboardInterrupt: 

In [23]:
model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1)
labeledQuestions = labeledQuestions1 + labeledQuestions2 + labeledQuestionstest1 + labeledQuestionstest2

NameError: name 'labeledQuestionstest2' is not defined

In [ ]:
model.build_vocab(labeledQuestions)

In [ ]:
model.train(labeledQuestions)
# for epoch in range(10):
#     model.train(labeledQuestions)
#     model.alpha -= 0.002  # decrease the learning rate`
#     model.min_alpha = model.alpha # fix the learning rate, no decay

In [ ]:
model.save('Models/4_22model_processed_combined')# 35900484 for raw

In [ ]:
model_loaded = models.Doc2Vec.load('Models/4_8model')

In [ ]:
similarity = []
for i in range(len(labeledQuestions1)):
    similarity.append(model.docvecs.similarity('question1%d'%i, 'question2%d'%i))

In [ ]:
similarityDF = pd.DataFrame({'similarity': similarity})
similarityDF['y'] = trainDF.is_duplicate

In [ ]:
vectors1 = []
vectors2 = []
print "Fetching Vectors..."
for i in range(len(labeledQuestions1)*2):
    if i< len(labeledQuestions1):
        vectors1.append(model.docvecs[i])
    else:
        vectors2.append(model.docvecs[i])
import ipdb; ipdb.set_trace()

vectors1 = np.asarray(vectors1)
vectors2 = np.asarray(vectors2)

labels = np.asarray(train.is_duplicate)


In [ ]:
vectorDiff = np.subtract(vectors1, vectors2)